In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/cv_project/wildanimal" -d /content/drive/MyDrive/wildanimal-dataset

Archive:  /content/drive/MyDrive/cv_project/wildanimal.zip
  inflating: /content/drive/MyDrive/wildanimal-dataset/Animals/Cheetah/animal-africa-wilderness-zoo.jpg  
  inflating: /content/drive/MyDrive/wildanimal-dataset/Animals/Cheetah/cheetah-1128798__340.jpg  
  inflating: /content/drive/MyDrive/wildanimal-dataset/Animals/Cheetah/cheetah-171217__340.jpg  
  inflating: /content/drive/MyDrive/wildanimal-dataset/Animals/Cheetah/cheetah-223734__340.jpg  
  inflating: /content/drive/MyDrive/wildanimal-dataset/Animals/Cheetah/cheetah-2268955__340.jpg  
  inflating: /content/drive/MyDrive/wildanimal-dataset/Animals/Cheetah/cheetah-2412554__340.jpg  
  inflating: /content/drive/MyDrive/wildanimal-dataset/Animals/Cheetah/cheetah-3099722__340.jpg  
  inflating: /content/drive/MyDrive/wildanimal-dataset/Animals/Cheetah/cheetah-322310__340.jpg  
  inflating: /content/drive/MyDrive/wildanimal-dataset/Animals/Cheetah/cheetah-3475778__340.jpg  
  inflating: /content/drive/MyDrive/wildanimal-dataset

In [ ]:
!unzip "/content/drive/MyDrive/cv_project/animals" -d /content/drive/MyDrive/wildanimal-dataset

Archive:  /content/drive/MyDrive/cv_project/animals.zip
   creating: /content/drive/MyDrive/wildanimal-dataset/archive (5)/
  inflating: /content/drive/MyDrive/wildanimal-dataset/archive (5)/.DS_Store  
   creating: /content/drive/MyDrive/wildanimal-dataset/archive (5)/cheetah/
  inflating: /content/drive/MyDrive/wildanimal-dataset/archive (5)/cheetah/.DS_Store  
 extracting: /content/drive/MyDrive/wildanimal-dataset/archive (5)/cheetah/00000000_224resized.png  
 extracting: /content/drive/MyDrive/wildanimal-dataset/archive (5)/cheetah/00000001_224resized.png  
  inflating: /content/drive/MyDrive/wildanimal-dataset/archive (5)/cheetah/00000002_224resized.png  
  inflating: /content/drive/MyDrive/wildanimal-dataset/archive (5)/cheetah/00000003_224resized.png  
 extracting: /content/drive/MyDrive/wildanimal-dataset/archive (5)/cheetah/00000004_224resized.png  
  inflating: /content/drive/MyDrive/wildanimal-dataset/archive (5)/cheetah/00000005_224resized.png  
 extracting: /content/drive/

In [2]:
from pathlib import Path
from PIL import Image
import shutil
import re

source_root1 = Path("/content/drive/MyDrive/wildanimal-dataset/Animals")
source_root2 = Path("/content/drive/MyDrive/wildanimal-dataset/animals2")
output_images = Path("/content/drive/MyDrive/yolo-data/images/train")
output_labels = Path("/content/drive/MyDrive/yolo-data/labels/train")

# Class map in lowercase
CLASS_MAP = {
    'cheetah': 4,
    'lion': 5,
    'tiger': 6,
    'fox': 7,
    'wolf': 8,
    'hyena': 9
}

def get_next_index(class_name, image_dir):
    pattern = re.compile(rf"{class_name}_(\d+)\.jpg")
    indices = [int(m.group(1)) for f in image_dir.glob(f"{class_name}_*.jpg") if (m := pattern.match(f.name))]
    return max(indices, default=0) + 1

# Source directories to process
source_roots = [source_root1, source_root2]

# Supported extensions
supported_exts = ["*.jpg", "*.jpeg", "*.png"]

for cls_lower in CLASS_MAP:
    class_id = CLASS_MAP[cls_lower]
    next_idx = get_next_index(cls_lower, output_images)

    for root in source_roots:
        class_folder = root / cls_lower
        if not class_folder.exists():
            continue

        # Get all supported image files
        image_paths = []
        for ext in supported_exts:
            image_paths.extend(class_folder.glob(ext))

        for img_path in image_paths:
            base = f"{cls_lower}_{next_idx:03d}"
            img_dest = output_images / f"{base}.jpg"
            label_dest = output_labels / f"{base}.txt"

            try:
                # Convert and save as .jpg
                with Image.open(img_path) as im:
                    im.convert("RGB").save(img_dest, "JPEG")

                # Write YOLO label (full-image bbox for now)
                label_line = f"{class_id} 0.5 0.5 1.0 1.0"
                with open(label_dest, "w") as f:
                    f.write(label_line)

                next_idx += 1

            except Exception as e:
                print(f"⚠️ Skipped file {img_path}: {e}")

⚠️ Skipped file /content/drive/MyDrive/wildanimal-dataset/animals2/cheetah/00000244_224resized.png: cannot identify image file '/content/drive/MyDrive/wildanimal-dataset/animals2/cheetah/00000244_224resized.png'


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


⚠️ Skipped file /content/drive/MyDrive/wildanimal-dataset/animals2/hyena/00000224_224resized.png: cannot identify image file '/content/drive/MyDrive/wildanimal-dataset/animals2/hyena/00000224_224resized.png'


In [3]:
!ls /content/drive/MyDrive/yolo-data/images/train | head -5

buffalo_001.jpg
buffalo_002.jpg
buffalo_003.jpg
buffalo_004.jpg
buffalo_005.jpg
